### Utilisation du TensorBoard Callback 

In [ ]:
import keras
from keras import layers, models
from keras.datasets import imdb
from keras.utils import pad_sequences

In [ ]:
max_features = 2000 
max_len = 500 

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, 500, 128)          256000    
                                                                 
 conv1d_2 (Conv1D)           (None, 494, 32)           28704     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 98, 32)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 1)                

In [ ]:
callbacks = [keras.callbacks.TensorBoard(log_dir='logs',histogram_freq=1,
                                         embeddings_freq=1,)]

history = model.fit(x_train, y_train,epochs=100,batch_size=128,
                    validation_split=0.2,callbacks=callbacks)

Epoch 1/100
157/157 [==============================] - 3s 14ms/step - loss: 0.5131 - acc: 0.7333 - val_loss: 0.3569 - val_acc: 0.8420
Epoch 2/100
157/157 [==============================] - 2s 12ms/step - loss: 0.3214 - acc: 0.8652 - val_loss: 0.4596 - val_acc: 0.8018
Epoch 3/100
157/157 [==============================] - 2s 11ms/step - loss: 0.2794 - acc: 0.8831 - val_loss: 0.3859 - val_acc: 0.8326
Epoch 4/100
157/157 [==============================] - 2s 12ms/step - loss: 0.2442 - acc: 0.9028 - val_loss: 0.3175 - val_acc: 0.8686
Epoch 5/100
157/157 [==============================] - 2s 13ms/step - loss: 0.2124 - acc: 0.9187 - val_loss: 0.3242 - val_acc: 0.8660
Epoch 6/100
157/157 [==============================] - 2s 12ms/step - loss: 0.1801 - acc: 0.9331 - val_loss: 0.5315 - val_acc: 0.8006
Epoch 7/100
157/157 [==============================] - 2s 13ms/step - loss: 0.1474 - acc: 0.9478 - val_loss: 0.3833 - val_acc: 0.8576
Epoch 8/100
157/157 [==============================] - 2s 12ms

##### Pour lancer Tensorboard il suffit d'exécuter cette commande dans un terminal: tensorboard --logdir emplacement du fichier contenant les logs

In [ ]:
tensorboard --logdir logs

##### Puis aller sur le lien spécifié par tensorboard, par exemple http://localhost:6006/

### Keras Tuner

In [ ]:
import kerastuner as kt
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import optimizers, layers, Sequential


(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


def build_model(hp):
    network = Sequential()
    
    for i in range(hp.Int('num_layers', 1,3)):
        network.add(layers.Dense(units = hp.Int(f'units_{i}',min_value = 256, max_value= 512, step=128),
                                activation = hp.Choice('activation', ['relu', 'tanh'])))
    
    network.add(layers.Dense(10, activation='softmax'))
    lr = hp.Choice('lr', [0.01,0.001,0.0001])
    network.compile(optimizer = optimizers.RMSprop(learning_rate = lr), loss = 'categorical_crossentropy', metrics=['acc'])
    
    return network

In [ ]:
tuner = kt.RandomSearch(build_model, objective='val_acc',executions_per_trial=1,max_trials=100, directory='tuner')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 256, 'max_value': 512, 'step': 128, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
lr (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(train_images, train_labels,validation_split=0.2, epochs=5)


Trial 100 Complete [00h 00m 43s]
val_acc: 0.9614999890327454

Best val_acc So Far: 0.9779999852180481
Total elapsed time: 01h 08m 05s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_model = tuner.get_best_models()

best_hyperparameters = tuner.get_best_hyperparameters()

In [ ]:
tuner.results_summary()

Results summary
Results in tuner_\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
units_0: 384
activation: relu
lr: 0.001
units_1: 384
units_2: 384
Score: 0.9779999852180481
Trial summary
Hyperparameters:
num_layers: 2
units_0: 384
activation: relu
lr: 0.001
units_1: 256
units_2: 256
Score: 0.9779999852180481
Trial summary
Hyperparameters:
num_layers: 1
units_0: 512
activation: relu
lr: 0.001
units_1: 384
units_2: 384
Score: 0.9776666760444641
Trial summary
Hyperparameters:
num_layers: 2
units_0: 512
activation: tanh
lr: 0.001
units_1: 384
units_2: 256
Score: 0.9775833487510681
Trial summary
Hyperparameters:
num_layers: 2
units_0: 512
activation: relu
lr: 0.001
units_1: 384
units_2: 512
Score: 0.9774166941642761
Trial summary
Hyperparameters:
num_layers: 2
units_0: 256
activation: relu
lr: 0.001
units_1: 384
units_2: 384
Score: 0.9769166707992554
Trial summary
Hyperparameters:
num_layers: 2
units_0: 512
activation: relu
lr: 0.001
units_1: 384
units_

In [ ]:
model = build_model(best_hyperparameters[0])

In [ ]:
history = model.fit(train_images, train_labels, epochs=10, validation_split = 0.2)

Epoch 1/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2112 - acc: 0.9366 - val_loss: 0.1254 - val_acc: 0.9624
Epoch 2/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0986 - acc: 0.9722 - val_loss: 0.1112 - val_acc: 0.9731
Epoch 3/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0748 - acc: 0.9806 - val_loss: 0.1198 - val_acc: 0.9746
Epoch 4/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0613 - acc: 0.9851 - val_loss: 0.1265 - val_acc: 0.9734
Epoch 5/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0532 - acc: 0.9866 - val_loss: 0.1437 - val_acc: 0.9741
Epoch 6/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0463 - acc: 0.9890 - val_loss: 0.1519 - val_acc: 0.9768
Epoch 7/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0392 - acc: 0.9907 - val_loss: 0.2057 - val_acc: 0.9758
Epoch 8/10
1500/1500 [==============================] - 7s 5ms